# <span style="color:gray">Pangolin SARS-CoV-2 Pipeline Notebook</span>

## Overview

We are going to run a standard covid bioinformatics pipeline using the [Pangolin workflow](https://cov-lineages.org/resources/pangolin/usage.html). We will run the whole analysis within this notebook environment.

## Learning Objectives
Learn how to run a simple bioinformatic workflow within a Jupyter notebook environment on AWS.

## Prerequisites
+ You only need access to a Sagemaker environment to run this notebook

## Get Started

### Install packages and set up environment

In [ ]:
import os

CPU = os.cpu_count()
print(f"Number of threads available: {CPU}")


In [ ]:
# install biopython to import packages below
! pip install biopython

In [ ]:
# install mamba
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge

In [ ]:
# add to your path
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

In [ ]:
# install everything else
! mamba install -y -c conda-forge -c bioconda sra-tools  pangolin iqtree

In [ ]:
#import libraries
import os
from Bio import SeqIO
from Bio import Entrez

### Set up your directory structure and remove files from previous runs if they exist

In [ ]:
if not os.path.exists('pangolin_analysis'):
    os.mkdir('pangolin_analysis')

os.chdir('pangolin_analysis')

In [ ]:
if os.path.exists('sarscov2_sequences.fasta'):
    os.remove('sarscov2_sequences.fasta')

! rm sarscov2_*
! rm lineage_report.csv

### Fetch viral sequences using a list of accession IDs

In [ ]:
#give a list of accession number for sars sequences
acc_nums=['NC_045512','LR757995','LR757996','OL698718','OL677199','OL672836','MZ914912','MZ916499','MZ908464','MW580573','MW580574','MW580576','MW991906','MW931310','MW932027','MW424864','MW453109','MW453110']

print('the number of sequences we will analyze = ',len(acc_nums))

Let this block run without going to the next until it finishes, otherwise you may get an error about too many requests. If that happens, reset your kernel and just rerun everything (except installing software).

In [ ]:
# use the bio.entrez toolkit within biopython to download the accession numbers
# save those sequences to a single fasta file
Entrez.email = "email@example.com"  # ell NCBI who you are

filename = "sarscov2_seqs.fasta"

if not os.path.isfile(filename):
    # Downloading...
    for acc in acc_nums:
        net_handle = Entrez.efetch(
            db="nucleotide", id=acc, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "a")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        print("Saved",acc)

In [ ]:
# make sure our fasta file has the same number of seqs as the acc_nums list

print('the number of seqs in our fasta file: ')

! grep '>' sarscov2_seqs.fasta | wc -l

In [ ]:
# let's peek at our new fasta file
! head sarscov2_seqs.fasta

### Run pangolin to identify lineages and output alignment
Here we call pangolin, give it our input sequences and the number of threads. We also tell it to output the alignment. The full list of pangolin parameters can be found in the pangolin [docs](https://cov-lineages.org/resources/pangolin/usage.html).

In [ ]:
! pangolin ./pangolin_analysis/sarscov2_seqs.fasta --threads $CPU

You can view the output file from pangolin called lineage_report.csv (within pangolin_analysis folder) by double clicking on the file, or by right clicking and downloading. What lineages are present in the dataset? Is Omicron in there?

### Run iqtree to estimate maximum likelihood tree for our sequences
iqtree can find the best nucleotide model for the data, but here we are going to assign a model to save time (HKY) and just estimate the phylogeny without any bootstrap support values. 

In [ ]:
# run iqtree with threads = $CPU variable.
# if you exclude the -m it will do a phylogenetic model search before tree search
! iqtree -s sequences.aln.fasta -nt $CPU -m HKY --prefix sarscov2_tree --redo-tree

### Download the tree and view in tree viewer like [FigTree](http://tree.bio.ed.ac.uk/software/figtree/)! 

### Conclusions
That's it! Now you know how to run a simple workflow using a Sagemaker notebook environment

## Clean up
Make sure you shut down this VM, or delete it if you don't plan to use if further.

You can also [delete the buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/delete-bucket.html) if you don't want to pay for the data: `aws s3 rb s3://bucket-name --force`